# 画像分類の実践
本稿ではオリジナルのデータセットを使用して画像分類を実践します。  ぜひソースコードを改変してより精度の高いモデルを学習できることを目標にいろいろ試してみてください。  
研修中ではデータセット「**車**」、「**猫**」、「**家**」の３クラス分類を行います。各クラス学習用100枚、テスト用10枚で実践します。機械学習ライブラリは**Pytorch**です。

### 事前準備
##### ・GPUの利用設定
ツールバーの"**ランタイム**"→"**ランタイムのタイプを変更**"→ハードウェアアクセラレータの"**GPU**"を選択→"**保存**"  
##### ・Google Driveのマウント
左にあるツールバー内の"**ファイル**"→"**ドライブをマウント**"  
（Google Drive上のファイルを読み書きできるようにするためにマウントを行います。）





### 注意
"Google Drive"のMyDrive配下にzip展開した"img_classification"フォルダが正しく配置されていることを確認してください。配置場所が間違えていると実行できません。  
  
例：colaboratory上のファイル配置  
/content/drive/MyDrive ─ img_classification ─ classify_demo.ipynb  
　　　　　　　　　　　　　　　　　　　　└　data 
　　　　　　　　　　　　 

# 手順１  
画像データセットを収集  
研修では「車」,「猫」,「家」の3クラス分類を行います。学習用100枚、テスト用10枚で実践します。

# 手順２
画像データセットを'data'フォルダ内に配置。下記のディレクトリ階層図を参考にしてください。  
デフォルトでは研修で使用するデータセットが既に配置済みです。
  
data ─ train ─ car ─ ＊.jpg    
　　│　　  │  
　　│　  　├ cat ─ ＊.jpg  
　　│　　  │  
　　│　  　└ house ─ ＊.jpg    
　　│  
　  　└ test ─ car ─ ＊.jpg  
　　　　  　 │   
　　　  　　 ├ cat ─ ＊.jpg  
　　 　　  　│  
　　　  　　 └ house ─ ＊.jpg    


# 手順３
下記のセル群を順番に実行

In [ ]:
import os
import copy
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
#from tqdm import tqdm
from tqdm.notebook import tqdm

# gpuが使用可能ならgpuを使用、不可能ならcpuを使用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### 学習用データセットの前処理
1.画像のリサイズ  
2.テンソル型に変換(pytorch特有の処理)  
3.標準化  
4.ミニバッチ化  

In [ ]:
# データカスタマイズセル
TRAIN_DIR = '/content/drive/MyDrive/img_classification/data/train'
BATCH_SIZE = 4

In [ ]:
train_transforms = transforms.Compose([
                                # 1.画像のリサイズ
                                transforms.Resize((224, 224)),
                                # 2.テンソル型に変換
                                transforms.ToTensor(),  
                                # 3.標準化
                                transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225])
                                ])

# 前処理したデータセットを読み込む
train_datasets = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)

# 4.ミニバッチ化
train_loaders = DataLoader(train_datasets, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

# 学習画像の枚数を取得
train_sizes = len(train_datasets)

# 学習データセットのクラスを取得
train_class_names = train_datasets.classes

In [ ]:
print(train_sizes)
print(train_class_names)

### データの中身を確認
上記セルの”train_loaders”(イテレータ)に格納したデータセット(1ミニバッチ)の中身を確認します。  
  
注意：テンソル型に変換された状態なのでnumpyで次元(軸)を変換するのと、標準化されたデータを元に戻すことが必要です。  
  
1.”train_loader”から画像とラベルを順番に取り出す  
2.テンソル型からnumpyの型に変換  
3.標準化されたデータを元に戻す

In [ ]:
def tensor_to_numpy(inp):
  # 2.テンソル型からnumpyの型に変換
  inp = inp.numpy().transpose((1,2,0))
  # 3.標準化されたデータを元に戻す
  mean = np.array([0.485, 0.456, 0.406])
  std = np.array([0.229, 0.224, 0.225])
  inp = std * inp + mean
  inp = np.clip(inp, 0, 1)
  return inp

# 1."train_loaders"から画像とラベルを順番に取り出す
inputs, classes = next(iter(train_loaders))
out = torchvision.utils.make_grid(inputs)

# imshowメソッドを実行
out = tensor_to_numpy(out)
plt.imshow(out)
plt.title([train_class_names[x] for x in classes])

### モデル　損失関数　最適化法の定義
転移学習のベースモデルは学習済みの**VGG16**を使用します。損失関数は**クロスエントロピー誤差**。最適化法は**MomentumSGD**を使用します。  
また今回は”torch.optim.lr_scheduler.StepLR”を使用して7epochごとに学習率を1/10にします。  
1.モデルの定義  
2.損失関数の定義  
3.最適化法の定義

In [ ]:
# 学習カスタマイズセル

# 1モデルの定義
model_ft = models.vgg16(pretrained=True)
num_ftrs = model_ft.classifier[6].in_features
# モデルの全結合層にある最終層のニューロン数をクラス数(研修では3クラス)に変更する
model_ft.classifier[6] = nn.Linear(num_ftrs, len(train_class_names))

# for ResNet18 (やってみよう2)
#model_ft = models.resnet18(pretrained=True)
#num_ftrs = model_ft.fc.in_features
#model_ft.fc = nn.Linear(num_ftrs, len(train_class_names))

model_ft = model_ft.to(device)

# 2.損失関数の定義
criterion = nn.CrossEntropyLoss()

# 3.最適化法の定義
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# エポックの指定
NUM_EPOCHS = 2

# 7epochごとに学習率を1/10にする(7エポック未満の場合は関係ない)
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

### 学習
1.モデルをトレーニングモードにする  
2.学習データとラベルを順番にとりだす  
3.勾配(微分)をリセット  
4.順伝播  
5.逆伝播  

In [ ]:
# 学習メソッドを宣言
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 0.0
    lossdata = []
    accdata = []
    
    for epoch in tqdm(range(num_epochs)):
        print('Epoch {}/{}'.format(epoch, num_epochs-1))
        print('-' * 10)
        
        scheduler.step()

        # 1.モデルをトレーニングモードにする
        model.train()

        running_loss = 0.0     # 損失関数の合計を保存するための変数
        running_corrects = 0   # 精度の合計を保存するための変数
            
        # 2.学習データとラベルを順番に取り出す
        for inputs, labels in train_loaders:
            inputs = inputs.to(device)      # 画像データを格納
            labels = labels.to(device)      # ラベル（教師データ）を格納
                
            # 3.勾配(微分)をリセット
            optimizer.zero_grad()
                
            # 4.順伝播
            outputs = model(inputs)      # 画像データをモデルに入力し、出力された予測データを取得
            _, preds = torch.max(outputs, 1)   # 予測データ(確率)の最大値を取得
            loss = criterion(outputs, labels)  # 予測データと教師データから誤差を取得
            
            # 5.逆伝播
            loss.backward()
            optimizer.step()   # 勾配(微分)計算
                
            # 損失関数と精度の合計を取得
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        # エポックごとの損失関数と精度を取得
        epoch_loss = running_loss / train_sizes
        epoch_acc = running_corrects.double() / train_sizes
            
        lossdata.append(epoch_loss)
        accdata.append(epoch_acc)

        if epoch_acc > best_acc:
            best_acc = epoch_acc
            # 精度が最も高いときに重みを格納
            best_model_wts = copy.deepcopy(model.state_dict())
            
        print('Loss: {:.4f} Acc: {:4f}'.format(epoch_loss, epoch_acc))
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best train Acc:{:4f}'.format(best_acc))

    # 最も精度が高かった重みを読み込む
    model.load_state_dict(best_model_wts)

    return model, lossdata, accdata


# training
model_ft, lossdata, accdata = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=NUM_EPOCHS)

### モデルの保存
model.state_dict()で保存が可能。重みのみを保存している。

In [ ]:
save_path = '/content/drive/MyDrive/img_classification/models'
os.makedirs(save_path, exist_ok=True)
torch.save(model_ft.state_dict(), os.path.join(save_path, 'model.pth'))

### モデルの読み込み
上記で重みのみを保存したモデルを読み込む際は、再度モデルを定義し直してから読み込む。

In [ ]:
# for vgg16
model = models.vgg16()
num_ftrs = model.classifier[6].in_features
# モデルの全結合層にある最終層のニューロン数をクラス数(研修では3クラス)に変更する
model.classifier[6] = nn.Linear(num_ftrs, len(train_class_names))

# for ResNet18
#model = models.resnet18()
#num_ftrs = model.fc.in_features
#model.fc = nn.Linear(num_ftrs, len(train_class_names))

model.load_state_dict(torch.load(os.path.join(save_path, 'model.pth')))

### 精度と損失関数のグラフ

In [ ]:
fig = plt.figure()

# 横軸の設定
x = list(range(NUM_EPOCHS))

# 精度グラフ
np_acc = np.array(accdata[:NUM_EPOCHS])
ax_acc = fig.add_subplot(1, 2, 1, title='train_acc', ylim=(0,1))
ax_acc.plot(x, np_acc)

# 損失関数グラフ
np_loss = np.array(lossdata[:NUM_EPOCHS])
ax_loss = fig.add_subplot(1, 2, 2, title='train_loss', ylim=(0,1))
ax_loss.plot(x, np_loss)

# グラフ出力
plt.show()

### テスト用データセットの前処理
1.画像のリサイズ  
2.テンソル型に変換(pytorch特有の処理)  
3.標準化  
4.ミニバッチ化  

In [ ]:
# 学習用データカスタマイズセル
TEST_DIR = '/content/drive/MyDrive/img_classification/data/test'

In [ ]:
TEST_DIR = os.path.join(TEST_DIR)
transform = transforms.Compose([
                    # 1.画像のリサイズ
                    transforms.Resize((224,224)),
                    # 2.テンソル型に変換
                    transforms.ToTensor(),
                    # 3.標準化 
                    transforms.Normalize([0.485, 0.456, 0.406],
                                        [0.229, 0.224, 0.225])
                    ])

# 前処理したデータセットを読み込む
test_datasets = datasets.ImageFolder(TEST_DIR, transform=transform)

# 4.ミニバッチ化
test_loaders = DataLoader(test_datasets, batch_size=1, shuffle=True, num_workers=0)

# テスト用画像の枚数を取得
test_sizes = len(test_datasets)
test_class_names = test_datasets.classes

### テスト
学習済みモデルを使って推論を行います。  
1.モデルを評価モードにする  
2.テストデータとラベルを順番に取り出す  
3.テストデータを読み込み、予測データを取得  
4.予測データの最大値を予測結果とする

In [ ]:
def visualize_model(model, num_images=0):
    # 1.モデルを評価モードにする
    model.eval()
    fig = plt.figure()
    acc = 0
    
    with torch.no_grad():
        # 2.テストデータとラベルを順番に取り出す
        for i, (inputs, labels) in enumerate(test_loaders):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # 3.テストデータを読み込み、予測データを取得
            outputs = model(inputs)

            # 4.予測データの最大値を予測結果とする
            _, preds = torch.max(outputs, 1)

            print('predicted: {} ,  label: {}'.format(train_class_names[preds], test_class_names[labels]))
            plt.imshow(tensor_to_numpy(inputs.cpu().data[0]))
            plt.show()
            
            if test_class_names[preds] == test_class_names[labels]:
                acc += 1

            print('-' * 60)
        print('{}枚中、{}枚正解'.format(str(num_images), str(acc)))
        model.train()


visualize_model(model_ft, len(test_loaders))

## やってみよう  
####１．自分の好きなオリジナルデータセットを使って画像分類を実践してみよう  
オリジナルデータセットを使って実践する方法は本稿の**手順1**と**手順2**で説明済みです。  
**<補足>**  
データセットは画像検索で集めることが可能ですが一枚ずつ保存していては大変です。Google Chromeユーザ限定の紹介になってしまいますが、**ImageDownloader**というChromeの拡張機能を使ってデータを集めることを推奨します。**ImageDownloader**のインストール方法は[こちら](https://chrome.google.com/webstore/detail/image-downloader/cnpniohnfphhjihaiiggeabnkjhpaldj?hl=ja&)  
  
####２．転移学習のベースモデルを**VGG16**から**ResNet18**に変更して学習させてみよう  
VGG16とResNet18の両方の結果を比較してみてください。研修で利用した「猫」「車」「家」の3クラス分類では難易度が低すぎるため違いが分かりにくいですが、10クラス以上の分類や細かな種類の分類問題を学習させてみると違いが見えてくると思います。  
ソースコードの改変箇所は本稿の**モデル 損失関数 最適化法の定義**にある**学習カスタマイズセル**の中身です。L4,5,7をコメントアウトして、L10,11,12の**#**を削除するとベースモデルをResNet18に変更できます。